# LQR appy to a Robot Arm in MuJoCo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mujoco
import mujoco.viewer as viewer
import mediapy as media
import scipy

In [3]:
class ModelPredictiveControl(object):
    
    # A,B,C - system matrices
    # f -  prediction horizon
    # v  - control horizon
    # W3 - input weight matrix
    # W4 - prediction weight matrix
    # x0 - initial state of the system
    # desiredControlTrajectoryTotal - total desired control trajectory
    #                               later on, we will take segments of this
    #                               desired state trajectory
    
    def __init__(self,A,B,C,f,v,W3,W4,x0,desiredControlTrajectoryTotal):
        
        # initialize variables
        self.A=A 
        self.B=B
        self.C=C
        self.f=f
        self.v=v
        self.W3=W3 
        self.W4=W4
        self.desiredControlTrajectoryTotal=desiredControlTrajectoryTotal
        
        # dimensions of the matrices
        
        self.n=A.shape[0]
        self.r=C.shape[0]
        self.m=B.shape[1]        
                
               
        # this variable is used to track the current time step k of the controller
        # after every calculation of the control inpu, this variables is incremented for +1 
        self.currentTimeStep=0
        
        # we store the state vectors of the controlled state trajectory
        self.states=[]
        self.states.append(x0)
        
        # we store the computed inputs 
        self.inputs=[]
        
        # we store the output vectors of the controlled state trajectory
        self.outputs=[]
        
        
        # form the lifted system matrices and vectors
        # the gain matrix is used to compute the solution 
        # here we pre-compute it to save computational time
        self.O, self.M, self.gainMatrix = self.formLiftedMatrices()
        
    
    # this function forms the lifted matrices O and M, as well as the 
    # the gain matrix of the control algorithm
    # and returns them 
    def formLiftedMatrices(self):
        f=self.f
        v=self.v
        r=self.r
        n=self.n
        m=self.m
        A=self.A
        B=self.B
        C=self.C
        
        # lifted matrix O
        O=np.zeros(shape=(f*r,n))

        for i in range(f):
            if (i == 0):
                powA=A;
            else:
                powA=np.matmul(powA,A)
            O[i*r:(i+1)*r,:]=np.matmul(C,powA)

        # lifted matrix M        
        M=np.zeros(shape=(f*r,v*m))
    
        for i in range(f):
            # until the control horizon
            if (i<v):
                for j in range(i+1):
                    if (j == 0):
                        powA=np.eye(n,n);
                    else:
                        powA=np.matmul(powA,A)
                    M[i*r:(i+1)*r,(i-j)*m:(i-j+1)*m]=np.matmul(C,np.matmul(powA,B))
            
            # from control horizon until the prediction horizon
            else:
                for j in range(v):
                    # here we form the last entry
                    if j==0:
                        sumLast=np.zeros(shape=(n,n))
                        for s in range(i-v+2):
                            if (s == 0):
                                powA=np.eye(n,n);
                            else:
                                powA=np.matmul(powA,A)
                            sumLast=sumLast+powA
                        M[i*r:(i+1)*r,(v-1)*m:(v)*m]=np.matmul(C,np.matmul(sumLast,B))
                    else:
                        powA=np.matmul(powA,A)
                        M[i*r:(i+1)*r,(v-1-j)*m:(v-j)*m]=np.matmul(C,np.matmul(powA,B))
        
        
        tmp1=np.matmul(M.T,np.matmul(self.W4,M))
        tmp2=np.linalg.inv(tmp1+self.W3)
        gainMatrix=np.matmul(tmp2,np.matmul(M.T,self.W4))
        
        
        return O,M,gainMatrix
    
    # this function propagates the dynamics
    # x_{k+1}=Ax_{k}+Bu_{k}
    
    def propagateDynamics(self,controlInput,state):

        xkp1=np.zeros(shape=(self.n,1))
        yk=np.zeros(shape=(self.r,1))
        xkp1=np.matmul(self.A,state)+np.matmul(self.B,controlInput)
        yk=np.matmul(self.C,state)
        
        return xkp1,yk
        
    # this function computes the control inputs, applies them to the system 
    # by calling the propagateDynamics() function and appends the lists
    # that store the inputs, outputs, states
    def computeControlInputs(self):
                
        # extract the segment of the desired control trajectory
        desiredControlTrajectory=self.desiredControlTrajectoryTotal[self.currentTimeStep:self.currentTimeStep+self.f,:]

        # compute the vector s
        vectorS=desiredControlTrajectory-np.matmul(self.O,self.states[self.currentTimeStep])
       
        # compute the control sequence
        inputSequenceComputed=np.matmul(self.gainMatrix,vectorS)
        inputApplied=np.zeros(shape=(1,1))
        inputApplied[0,0]=inputSequenceComputed[0,0]
        
        # compute the next state and output
        state_kp1,output_k=self.propagateDynamics(inputApplied,self.states[self.currentTimeStep])
        
        # append the lists
        self.states.append(state_kp1)
        self.outputs.append(output_k)
        self.inputs.append(inputApplied)
        # increment the time step
        self.currentTimeStep=self.currentTimeStep+1
        

## Load Model

In [2]:
# load model
xml = "../mujoco_menagerie/universal_robots_ur5e/scene.xml"
model = mujoco.MjModel.from_xml_path(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

## Render random noise

In [3]:
DURATION  = 5   # seconds
FRAMERATE = 60  # Hz

# Make a new camera, move it to a closer distance.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 1

# Initialize the robot pose.
mujoco.mj_resetDataKeyframe(model, data, 1)

frames = []
while data.time < DURATION:
    # Set control vector.
    data.ctrl = np.random.randn(model.nu)
    
    # Step the simulation.
    mujoco.mj_step(model, data)    

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        # # Set the lookat point to the humanoid's center of mass.
        # camera.lookat = data.body('torso').subtree_com
        
        renderer.update_scene(data, camera)
        pixels = renderer.render()
        frames.append(pixels)

# Display video.
media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


## Define setpoint for end effector position

In [89]:
# method that give the end effector position and return the qpos
def inverse_kinematics(model, data, body_id, desire_point, delta_point, jacp, jacr):
    mujoco.mj_forward(model, data) # compute forward kinematics
    mujoco.mj_jac(model, data, jacp, jacr, desire_point, body_id) # calculate jacobians
    
    delta_pos = np.linalg.pinv(jacp) @ delta_point #calculate deltas of position
    data.qpos += delta_pos #update actuators position

# method to plot images
def plot_image(model, data, renderer):
    renderer.update_scene(data)
    media.show_image(renderer.render())
    
# Init variables.
body_id = 6
x, dx = 0.2, 0.5
y, dy = 0.5, 0.5
z, dz = 0.2, 0.5
desire_point = np.array([x, y, z])
delta_point = np.array([dx, dy, dz])
jacp = np.zeros((3, model.nv)) #translation jacobian
jacr = np.zeros((3, model.nv)) #rotational jacobian

#get desire point
mujoco.mj_resetDataKeyframe(model, data, 1)
inverse_kinematics(model, data, body_id, desire_point, delta_point,jacp, jacr)

#calculate the torques to mantain the end effector position
mujoco.mj_forward(model, data)
data.qacc = 0
qpos0 = data.qpos.copy()  # Save the position setpoint.
mujoco.mj_inverse(model, data) # compute actuator force
qfrc0 = data.qfrc_inverse.copy() # save actuactor forces for the position
print('desired forces:', qfrc0)

ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(data.actuator_moment)
ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
print('control setpoint:', ctrl0)

desired forces: [ 0.00000000e+00 -4.24547242e+01 -1.16985320e+01  1.09925180e-01
  1.69315684e-04 -2.77555756e-17]
control setpoint: [ 0.00000000e+00 -4.24547242e+01 -1.16985320e+01  1.09925180e-01
  1.69315684e-04 -2.77555756e-17]


In [90]:
DURATION  = 2   # seconds
FRAMERATE = 60  # Hz

# Make a new camera position #TODO:update this position to one better.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 2
camera.azimuth = -25

# Set the state and controls to their setpoints.
mujoco.mj_resetData(model, data)
data.qpos = qpos0
data.ctrl = ctrl0

frames = []
while data.time < DURATION:
  # Step the simulation.
    mujoco.mj_step(model, data)

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        # Set the lookat point to the humanoid's center of mass.
        # camera.lookat = data.body('torso').subtree_com
        renderer.update_scene(data, camera)
        pixels = renderer.render()
        frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)


This browser does not support the video tag.


## Design LQR controller

In [91]:
nu = model.nu  # Alias for the number of actuators.
R = np.eye(nu)
print(R)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [95]:
#TODO:design Q matrix
# Get all joint names.
joint_names = [model.joint(i).name for i in range(model.njnt)]

# Get indices of joints.
nv = model.nv
root_dofs = range(6)

all_dofs = [
    model.joint(name).dofadr[0]
    for name in joint_names
]

# print(root_dofs)
# Cost coefficients
COST = 1000

# Construct the Q matrix
Qjoint = np.eye(nv)
# print(Qjoint)

# Qjoint[all_dofs] *= COST

Qpos = 100 * Qjoint
# print(Qjoint)
# print(Qpos)

Q = np.block([[Qpos, np.zeros((nv, nv))],
              [np.zeros((nv, 2*nv))]])
print(Q)

[[100.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 100.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. 100.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0. 100.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 100.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. 100.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]


## Compute gain K

In [96]:
nu = model.nu # number of actuators
nv = model.nv  # number of degree of freedoms.

# set initial state and control
mujoco.mj_resetData(model, data)
data.ctrl = ctrl0
data.qpos = qpos0

# define the continuous-time system matrices A, B, C
A = np.zeros((2*nv, 2*nv))
B = np.zeros((2*nv, nu))

epsilon = 1e-6
flg_centered = False

mujoco.mjd_transitionFD(model, data, epsilon, flg_centered, A, B, None, None)

# Solve discrete Riccati equation.
P = scipy.linalg.solve_discrete_are(A, B, Q, R)

# Compute the feedback gain matrix K.
K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

print(K)

[[-1.05473329e+01  6.81573913e+02  3.80821288e+04 -1.08665055e+04
  -3.48059224e+03 -5.47202015e-02 -5.24438483e+01 -3.13085814e+02
   1.44796476e+04 -4.05218215e+03 -5.87357674e+03 -2.05361220e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-5.

## Render Test

In [97]:
# Parameters.
DURATION = 5         # seconds
FRAMERATE = 60        # Hz

# Reset data, set initial pose.
mujoco.mj_resetData(model, data)
data.qpos = qpos0

# Allocate position difference dq.
dq = np.zeros(model.nv)

# Make a new camera position #TODO:update this position to one better.
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 2
camera.azimuth = -25


frames = []
while data.time < DURATION:
    # Get state difference dx.
    mujoco.mj_differentiatePos(model, dq, 1, qpos0, data.qpos)
    dx = np.hstack((dq, data.qvel)).T

    # LQR control law.
    data.ctrl = ctrl0 - K @ dx

    # Step the simulation.
    mujoco.mj_step(model, data)

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.
